### Apriori
Apriori原理是说如果某个项集是频繁的，那么它的所有子集也是频繁的。这个原理直观上并没有什么帮助，但是如果反过来看就有用了，也就是说如果一个项集是非频繁集，那么它的所有超集也是非频繁的。 

### 生成候选项集
```
对数据集中的每条交易记录tran:
    对每个候选项集can:
        检查一下can是否是tran的子集:
            如果是，则增加can的计数值

对每个候选项集:
    如果其支持度不低于最小值，则保留该项集

返回所有频繁项集列表
```

### 组织完整的Apriori算法
```
当集合中项的个数大于0时:
    构建一个k个项组成的候选项集的列表
    检查数据以确认每个项集都是频繁的
    保留频繁项集并构建k+1项组成的候选项集的列表
```

In [1]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

# 对C1中每个项构建一个不变集合
def createC1(dataSet):
    C1 = []
    for transaction in dataSet: # 对数据集中的每条交易记录tran
        for item in transaction: # 对每个候选项集can
            if not [item] in C1: # 检查一下can是否是tran的子集
                C1.append([item]) # 如果是，则增加can的计数值
    C1.sort()
    return list(map(frozenset, C1))

# 计算所有项集的支持度
def scanD(D,Ck,minSupport):
    ssCnt={}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can]=1
                else: ssCnt[can]+=1
    numItems=float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt: # 对每个候选项集
        support = ssCnt[key] / numItems
        if support >= minSupport: # 如果其支持度不低于最小值，则保留该项集
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData # 返回所有频繁项集列表

In [2]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: # 前k-2个项相同时，将两个集合合并
                retList.append(Lk[i] | Lk[j])
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    # 扫描数据集，从Ck得到Lk
    while (len(L[k-2]) > 0): # 当集合中项的个数大于0时:
        Ck = aprioriGen(L[k-2], k) # 构建一个k个项组成的候选项集的列表
        Lk, supK = scanD(D, Ck, minSupport) # 检查数据以确认每个项集都是频繁的
        supportData.update(supK) # 保留频繁项集并构建k+1项组成的候选项集的列表
        L.append(Lk)
        k += 1
    return L, supportData

In [3]:
# 生成关联规则
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)): # 只获取有两个或更多元素的集合
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

# 生成候选规则集合
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H: # 遍历H中的所有项集并计算它们的可信度值
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            # print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

# 对规则进行评估
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): # 尝试进一步合并
        Hmp1 = aprioriGen(H, m+1) # 创建Hm+1条新候选规则
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [4]:
dataSet = loadDataSet()

In [5]:
L, suppData = apriori(dataSet, minSupport=0.5)
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [6]:
rules = generateRules(L, suppData, minConf=0.7)
rules

[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

In [7]:
rules = generateRules(L, suppData, minConf=0.5)
rules

[(frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

### 毒蘑菇数据集
数据集中包含了8124个样本和22个变量（如蘑菇的颜色、形状、光滑度等）。

每一种都被分为绝对可食用的，绝对有毒的，或者具有未知的可食性。

该数据集是探究机器学习分类算法算法不可多得的一个优质数据集。

parameters:
```
cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
bruises: bruises=t,no=f
odor: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s
gill-attachment: attached=a,descending=d,free=f,notched=n
gill-spacing: close=c,crowded=w,distant=d
gill-size: broad=b,narrow=n
gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
stalk-shape: enlarging=e,tapering=t
stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?
stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
veil-type: partial=p,universal=u
veil-color: brown=n,orange=o,white=w,yellow=y
ring-number: none=n,one=o,two=t
ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d
```

examples:
```
1 3 9 13 23 25 34 36 38 40 52 54 59 63 67 76 85 86 90 93 98 107 113 
2 3 9 14 23 26 34 36 39 40 52 55 59 63 67 76 85 86 90 93 99 108 114 
2 4 9 15 23 27 34 36 39 41 52 55 59 63 67 76 85 86 90 93 99 108 115 
```

In [8]:
mushDataSet = [line.split() for line in open('data/mushroom.dat').readlines()]

In [9]:
L, suppData = apriori(mushDataSet, minSupport=0.3)

In [10]:
for item in L[1]:
    if item.intersection('2'): # 有毒
        print(item, suppData[item])

frozenset({'28', '2'}) 0.4194977843426883
frozenset({'53', '2'}) 0.3190546528803545
frozenset({'23', '2'}) 0.33874938453963566
frozenset({'34', '2'}) 0.49433776464795665
frozenset({'36', '2'}) 0.3702609551944855
frozenset({'2', '59'}) 0.448055145248646
frozenset({'2', '63'}) 0.41851304775972425
frozenset({'67', '2'}) 0.33874938453963566
frozenset({'2', '76'}) 0.3328409650418513
frozenset({'2', '85'}) 0.517971442639094
frozenset({'2', '86'}) 0.49433776464795665
frozenset({'2', '90'}) 0.4529788281634663
frozenset({'2', '93'}) 0.3879862136878385
frozenset({'39', '2'}) 0.48252092565238797


In [11]:
for item in L[3]:
    if item.intersection('2'): # 有毒
        print(item, suppData[item])

frozenset({'34', '28', '53', '2'}) 0.3072378138847858
frozenset({'34', '28', '59', '2'}) 0.3259478089611029
frozenset({'34', '28', '63', '2'}) 0.32003938946331856
frozenset({'34', '28', '2', '85'}) 0.39586410635155095
frozenset({'34', '28', '2', '86'}) 0.39586410635155095
frozenset({'34', '28', '2', '90'}) 0.3308714918759232
frozenset({'28', '63', '59', '2'}) 0.30969965534219596
frozenset({'28', '59', '85', '2'}) 0.3495814869522403
frozenset({'28', '59', '86', '2'}) 0.3259478089611029
frozenset({'28', '59', '90', '2'}) 0.30428360413589367
frozenset({'28', '63', '2', '85'}) 0.3436730674544559
frozenset({'28', '63', '2', '86'}) 0.32003938946331856
frozenset({'28', '2', '86', '85'}) 0.39586410635155095
frozenset({'28', '85', '2', '90'}) 0.35450516986706054
frozenset({'28', '2', '86', '90'}) 0.3308714918759232
frozenset({'39', '28', '53', '2'}) 0.3072378138847858
frozenset({'39', '28', '34', '2'}) 0.3722304283604136
frozenset({'39', '28', '59', '2'}) 0.3289020187099951
frozenset({'39', '28

In [12]:
L, suppData = apriori(mushDataSet, minSupport=0.4)

In [13]:
for item in L[1]:
    if item.intersection('2'): # 有毒
        print(item, suppData[item])

frozenset({'28', '2'}) 0.4194977843426883
frozenset({'34', '2'}) 0.49433776464795665
frozenset({'2', '59'}) 0.448055145248646
frozenset({'2', '63'}) 0.41851304775972425
frozenset({'2', '85'}) 0.517971442639094
frozenset({'2', '86'}) 0.49433776464795665
frozenset({'2', '90'}) 0.4529788281634663
frozenset({'39', '2'}) 0.48252092565238797


In [14]:
for item in L[3]:
    if item.intersection('2'): # 有毒
        print(item, suppData[item])

frozenset({'34', '59', '2', '85'}) 0.4244214672575086
frozenset({'34', '59', '2', '86'}) 0.4244214672575086
frozenset({'34', '2', '86', '85'}) 0.49433776464795665
frozenset({'34', '85', '2', '90'}) 0.4293451501723289
frozenset({'34', '2', '86', '90'}) 0.4293451501723289
frozenset({'59', '2', '86', '85'}) 0.4244214672575086
frozenset({'59', '85', '2', '90'}) 0.40275726243229937
frozenset({'85', '2', '86', '90'}) 0.4293451501723289
frozenset({'39', '34', '2', '85'}) 0.4588872476612506
frozenset({'39', '34', '2', '86'}) 0.4588872476612506
frozenset({'39', '59', '85', '2'}) 0.4155588380108321
frozenset({'39', '2', '86', '85'}) 0.4588872476612506
frozenset({'39', '85', '2', '90'}) 0.4175283111767602


In [15]:
L, suppData = apriori(mushDataSet, minSupport=0.5)

In [16]:
for item in L[1]:
    if item.intersection('2'): # 有毒
        print(item, suppData[item])

frozenset({'2', '85'}) 0.517971442639094


In [17]:
for item in L[3]:
    if item.intersection('2'): # 有毒
        print(item, suppData[item])

接下来你需要观察一下这些特征，以便知道了解野蘑菇的那些方面。如果看到其中任何一个特征，那么这些蘑菇就不要吃了。

当然最后还是要声明一下：

    尽管上述这些特征在毒蘑菇中很普遍，但是没有这些特征并不意味着该蘑菇就是可食用的。如果吃错了蘑菇，你可能会因此而丧命。